<a href="https://colab.research.google.com/github/Amisha1019/Traslation-intership-of-nullclass/blob/main/Task_6_of_translation_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torchaudio gradio gtts sentencepiece --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.5 MB/s eta 0:00:00


In [5]:
import warnings
warnings.filterwarnings("ignore")


In [6]:
import gradio as gr
import torch
import os
from transformers import pipeline
from gtts import gTTS


In [4]:
asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    device=0 if torch.cuda.is_available() else -1
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [7]:
en_to_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
es_to_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [8]:
def speak(text, lang):
    tts = gTTS(text=text, lang=lang)
    filename = "output.mp3"
    tts.save(filename)
    return filename


In [9]:
def english_to_spanish(audio):
    if audio is None:
        return "No audio received", None

    # Speech to text
    result = asr(audio)
    english_text = result["text"]

    # Translate
    spanish = en_to_es(english_text)[0]["translation_text"]

    # Speak
    audio_file = speak(spanish, "es")

    return f"English: {english_text}\nSpanish: {spanish}", audio_file


In [10]:
def spanish_to_english(audio):
    if audio is None:
        return "No audio received", None

    # Speech to text
    result = asr(audio)
    spanish_text = result["text"]

    # Translate
    english = es_to_en(spanish_text)[0]["translation_text"]

    # Speak
    audio_file = speak(english, "en")

    return f"Spanish: {spanish_text}\nEnglish: {english}", audio_file


In [11]:
english_ui = gr.Interface(
    fn=english_to_spanish,
    inputs=gr.Audio(type="filepath", label="Speak English"),
    outputs=[
        gr.Textbox(label="Translation"),
        gr.Audio(label="Spanish Audio Output")
    ],
    title="English → Spanish Voice Translator"
)

spanish_ui = gr.Interface(
    fn=spanish_to_english,
    inputs=gr.Audio(type="filepath", label="Speak Spanish"),
    outputs=[
        gr.Textbox(label="Translation"),
        gr.Audio(label="English Audio Output")
    ],
    title="Spanish → English Voice Translator"
)


In [12]:
if __name__ == "__main__":
    gr.TabbedInterface(
        [english_ui, spanish_ui],
        ["English Speaker", "Spanish Speaker"]
    ).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://abffb6d590c7d4b5cc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
